<a href="https://colab.research.google.com/github/juridicodiniz/devai/blob/main/Atividade_5_Aprendizado_semi_supervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Aprendizado semi-supervisionado**

Usando o notebook em postre/aula9a_aprendizado_semi_supervisionado.ipynb at main · fboldt/postre (github.com) como inspiração, use outro algoritmo de agrupamento (clusterização), que não seja o KMeans, para escolher exemplos representativos da base mnist e para propagar os rótulos desses exemplos.

Compare o resultado do score de uma Regressão Logística com o dataset completo e com algumas amostras aleatórias.

In [1]:
from keras.datasets import mnist
import numpy as np

from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt



# Carregando o conjunto de dados MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Convertendo imagens 28x28 para vetores de 784 elementos
X_train = X_train.reshape(X_train.shape[0], -1).astype(np.float32)
X_test = X_test.reshape(X_test.shape[0], -1).astype(np.float32)

# Normalizando os dados
X_train /= 255.0


print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(y_test.shape))

11490434/11490434 [==============================] - 2s 0us/step
X_train: (60000, 784)
Y_train: (60000,)
X_test:  (10000, 784)
Y_test:  (10000,)


In [2]:
#Usando a regressão logistica
from sklearn.linear_model import LogisticRegression

#Supondo 50 y, que serão os 50  primerios
n_labeled = 50

#Interação maxima de 10 mil
log_reg = LogisticRegression(max_iter=10_000)

#Terinando a regressão Logistica
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

LogisticRegression(max_iter=10000)

In [3]:
log_reg.score(X_test, y_test)

0.493

In [4]:
#Usando o DBSCAN para agrupar e formar os clusters ******************************************
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

# Reduzindo a dimensionalidade com PCA para acelerar o DBSCAN
#usando 50 clustes
k = 50
pca = PCA(n_components=k)
X_pca = pca.fit_transform(X_train)


# Aplicando o DBSCAN
dbscan = DBSCAN(eps=5, min_samples=5)
labels = dbscan.fit_predict(X_pca)

X_mnist_dist = dbscan.fit_predict(X_pca)

#Pegando o indice dos que estão mais proximos do centro
representative_mnist_idx = X_mnist_dist.argmin(axis=0)

#Pegando os elementos dos indices dos que estão mais proximos do centro
X_representative_mnist = X_train[representative_mnist_idx]


In [5]:
# Visualizando alguns exemplos representativos de cada cluster com DBSCAN
unique_labels = set(labels)
representative_samples = []

for label in unique_labels:
    cluster_samples = X_pca[labels == label]
    representative_samples.append(cluster_samples[0])  # Escolhendo o primeiro exemplo de cada cluster
print(representative_samples)

[array([ 0.48594412, -1.2262338 , -0.09616432, -2.1794274 , -0.10709976,
       -0.91166735,  0.91762316,  0.6267464 , -1.4256616 ,  0.7781802 ,
        0.7745256 , -0.99639654, -0.4449332 ,  2.938374  ,  0.8599027 ,
       -0.01846454,  1.2947291 ,  1.2124268 ,  1.0886215 ,  0.6523089 ,
        0.10797346, -0.2523429 , -0.8516908 ,  0.9813607 ,  0.18109855,
        0.39338428, -1.2078079 ,  0.6669989 , -0.49216872, -0.48350343,
       -0.38451922,  0.13461949,  0.42194164,  0.3325395 , -0.5980506 ,
        0.78566885,  0.24960995,  0.7851004 ,  0.6135905 , -1.4137497 ,
        0.7311605 ,  0.5958584 , -0.15913379, -0.342273  , -0.47050828,
        0.04193472,  0.4791633 , -0.2720622 ,  0.199026  , -0.37880555],
      dtype=float32), array([ 1.5399216 ,  0.2837511 , -0.79433423,  0.7463034 ,  2.0303388 ,
       -1.18871   ,  1.7684039 , -0.7749004 ,  3.6898832 ,  1.744925  ,
        1.8745075 , -0.34054053, -0.04909857,  1.0707121 ,  0.6915199 ,
        1.0304608 ,  1.5076292 , -0.5443